In [1]:
import sys
sys.path.append('../..')

In [2]:
from src.recommenders.baseline_recommender import BaselineRecommender

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
books_df = pd.read_csv('../../data/Books.csv', delimiter=';')
ratings_df = pd.read_csv('../../data/Ratings.csv', delimiter=';')
users_df = pd.read_csv('../../data/Users.csv', delimiter=';')

/var/folders/s0/4m_t_82936ggv17n0g0t7_dr0000gn/T/ipykernel_38549/2786065433.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users_df = pd.read_csv('../../data/Users.csv', delimiter=';')


In [5]:
train_ratings, test_ratings = train_test_split(ratings_df, test_size=0.3, random_state=42)

In [6]:
model = BaselineRecommender()
model.fit(items=books_df, users=users_df, ratings=train_ratings)

In [7]:
test_users = test_ratings['User-ID'].unique()

In [8]:
best_items = model.predict(test_users)

In [9]:
best_items

[array(['0316779059', '0140268863', '1844262553', '3522128001',
        '0064440508', '8478886451', '020530902X', '0140126716',
        '8445071416', '0836213319'], dtype=object),
 array(['0316779059', '0140268863', '1844262553', '3522128001',
        '0064440508', '8478886451', '020530902X', '0140126716',
        '8445071416', '0836213319'], dtype=object),
 array(['0316779059', '0140268863', '1844262553', '3522128001',
        '0064440508', '8478886451', '020530902X', '0140126716',
        '8445071416', '0836213319'], dtype=object),
 array(['0316779059', '0140268863', '1844262553', '3522128001',
        '0064440508', '8478886451', '020530902X', '0140126716',
        '8445071416', '0836213319'], dtype=object),
 array(['0316779059', '0140268863', '1844262553', '3522128001',
        '0064440508', '8478886451', '020530902X', '0140126716',
        '8445071416', '0836213319'], dtype=object),
 array(['0316779059', '0140268863', '1844262553', '3522128001',
        '0064440508', '8478886451', 